In [1]:
import pandas as pd
from pyomo.environ import Set, SolverFactory, ConcreteModel, Constraint, Param, Objective, Var, Binary, minimize
from datetime import datetime, timedelta, time
import numpy as np

In [2]:
shipments = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_shipments')
df = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_schedule')
capacity = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_capacity')

In [3]:
def convert_string_to_list(s):
    # Replace colons with quotes to make it a valid Python expression
    s = s.replace(", ", ",").replace("[","").replace("]","")
    return s

df['new_departure']=df['scheduled_depart_weekday_time_local'].apply(convert_string_to_list)

In [4]:
def parse_new_departure(s):
    tuples = s.split('),(')
    tuples = [t.replace('(', '').replace(')', '') for t in tuples]
    parsed_tuples = [tuple(t.split(',')) for t in tuples]
    return parsed_tuples

In [5]:
# Apply the function to the new_departure column
df['parsed_departures'] = df['new_departure'].apply(parse_new_departure)

# Explode the parsed_departures column
schedule = df.explode('parsed_departures')

# Create the day_of_week and departure_time columns
schedule[['day_of_week', 'departure_time']] = pd.DataFrame(schedule['parsed_departures'].tolist(), index=schedule.index)

# Convert the day_of_week to integer
schedule['day_of_week'] = schedule['day_of_week'].astype(int)

# Drop the parsed_departures column
schedule['parsed_departure_time'] = schedule['departure_time'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

# Reset index if necessary
schedule.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
schedule.head(5)

,carrier_id,origin_region,destination_region,scheduled_depart_weekday_time_local,travel_hrs_with_timezone_offset,truck_type,rate_type,rate_break,new_departure,parsed_departures,day_of_week,departure_time,parsed_departure_time
0,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00
1,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(2, 20:00)",2,20:00,20:00:00
2,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00
3,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(4, 20:00)",4,20:00,20:00:00
4,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(5, 20:00)",5,20:00,20:00:00


In [6]:
#schedule['parsed_departure_dt']=np.where(schedule['day_of_week']==1, ['2024-05-06', '2024-05-13'],
                                         #np.where(schedule['day_of_week']==2, ['2024-05-07', '2024-05-14']),
                                         #         np.where(schedule['day_of_week']==3, ['2024-05-08', '2024-05-15']),
                                         #                  np.where(schedule['day_of_week']==4, ['2024-05-09', '2024-05-16']), 
                                         #                           np.where(schedule['day_of_week']==5, ['2024-05-10', '2024-05-17'])
                                         #)

#schedule['parsed_departure_dt']=['2024-05-06', '2024-05-13']

#schedule['parsed_departure_dt']=[['2024-05-06', '2024-05-13'] for i in schedule.index]
#schedule['parsed_departure_dt']=[['2024-05-07', '2024-05-14'] for i in schedule.index if schedule[i]['day_of_week']==2]

#schedule.loc[schedule['day_of_week']==2, 'parsed_departure_dt']=['2024-05-07', '2024-05-14']
#schedule.loc[schedule['day_of_week']==3, 'parsed_departure_dt']=['2024-05-08', '2024-05-15']
#schedule.loc[schedule['day_of_week']==4, 'parsed_departure_dt']=['2024-05-09', '2024-05-16']
#schedule.loc[schedule['day_of_week']==5, 'parsed_departure_dt']=['2024-05-10', '2024-05-17']

def apply_dates(dow):
    if dow==1:
        return ['2024-05-06', '2024-05-13']
    else: 
        return ['2024-05-10', '2024-05-17']

a=schedule.apply(lambda x: apply_dates(lambda x: schedule['day_of_week']))


In [7]:
max_dt=max(shipments['due_time_local'])
min_dt=min(shipments['ready_time_local'])-timedelta(days=5)
m=pd.date_range(min_dt, max_dt, freq='D')
df_dates = pd.DataFrame(m, columns=['date'])
df_dates['date_only']=df_dates['date'].dt.date
df_dates['dow'] = df_dates['date'].dt.dayofweek+1
df_dates

,date,date_only,dow
0,2024-05-04 10:00:00,2024-05-04,6
1,2024-05-05 10:00:00,2024-05-05,7
2,2024-05-06 10:00:00,2024-05-06,1
3,2024-05-07 10:00:00,2024-05-07,2
4,2024-05-08 10:00:00,2024-05-08,3
5,2024-05-09 10:00:00,2024-05-09,4
6,2024-05-10 10:00:00,2024-05-10,5
7,2024-05-11 10:00:00,2024-05-11,6
8,2024-05-12 10:00:00,2024-05-12,7
9,2024-05-13 10:00:00,2024-05-13,1


In [8]:
schedule=schedule.merge(df_dates, how='left', left_on='day_of_week', right_on='dow')
schedule['departure_dt_parsed']=pd.to_datetime(schedule['date_only'].astype(str) + ' ' + schedule['departure_time'])
schedule

,carrier_id,origin_region,destination_region,scheduled_depart_weekday_time_local,travel_hrs_with_timezone_offset,truck_type,rate_type,rate_break,new_departure,parsed_departures,day_of_week,departure_time,parsed_departure_time,date,date_only,dow,departure_dt_parsed
0,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00,2024-05-06 10:00:00,2024-05-06,1,2024-05-06 20:00:00
1,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00,2024-05-13 10:00:00,2024-05-13,1,2024-05-13 20:00:00
2,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(2, 20:00)",2,20:00,20:00:00,2024-05-07 10:00:00,2024-05-07,2,2024-05-07 20:00:00
3,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(2, 20:00)",2,20:00,20:00:00,2024-05-14 10:00:00,2024-05-14,2,2024-05-14 20:00:00
4,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00,2024-05-08 10:00:00,2024-05-08,3,2024-05-08 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1234,LAX,ATL,"[(1, 20:00), (3, 20:00), (5, 20:00)]",50,1,FTL,6000,"(1,20:00),(3,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00,2024-05-13 10:00:00,2024-05-13,1,2024-05-13 20:00:00
82,1234,LAX,ATL,"[(1, 20:00), (3, 20:00), (5, 20:00)]",50,1,FTL,6000,"(1,20:00),(3,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00,2024-05-08 10:00:00,2024-05-08,3,2024-05-08 20:00:00
83,1234,LAX,ATL,"[(1, 20:00), (3, 20:00), (5, 20:00)]",50,1,FTL,6000,"(1,20:00),(3,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00,2024-05-15 10:00:00,2024-05-15,3,2024-05-15 20:00:00
84,1234,LAX,ATL,"[(1, 20:00), (3, 20:00), (5, 20:00)]",50,1,FTL,6000,"(1,20:00),(3,20:00),(5,20:00)","(5, 20:00)",5,20:00,20:00:00,2024-05-10 10:00:00,2024-05-10,5,2024-05-10 20:00:00


In [9]:
# Function to parse rate breaks
def parse_rate_break(rate_break):
    if isinstance(rate_break, str):
        rate_break = eval(rate_break)
    if isinstance(rate_break, list):
        rate_break = rate_break[0]
    return rate_break

In [10]:
schedule['parsed_rate_break'] = schedule['rate_break'].apply(parse_rate_break)

In [11]:
schedule['parsed_rate_break'][5]

5000

## MODEL

In [12]:
model = ConcreteModel()

In [13]:
model.SHIPMENTS = Set(initialize=shipments['shipment_id'].tolist())
model.LINEHAULS = Set(initialize=schedule.index.tolist())

SHIPMENT PARAMETERS

In [14]:
# SHIPMENT Parameters
shipment_weight = dict(zip(shipments['shipment_id'], shipments['weight_kg']))
shipment_pallet = dict(zip(shipments['shipment_id'], shipments['pallet']))
shipment_ready_time = dict(zip(shipments['shipment_id'], pd.to_datetime(shipments['ready_time_local'])))
shipment_due_time = dict(zip(shipments['shipment_id'], pd.to_datetime(shipments['due_time_local'])))
#shipment_ready_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['ready_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))
#shipment_due_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['due_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))


MODEL PARAMETERS

In [15]:
# Parameters in the model
model.weight = Param(model.SHIPMENTS, initialize=shipment_weight)
model.pallet = Param(model.SHIPMENTS, initialize=shipment_pallet)
model.ready_time = Param(model.SHIPMENTS, initialize=shipment_ready_time)
model.due_time = Param(model.SHIPMENTS, initialize=shipment_due_time)
model.travel_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['travel_hrs_with_timezone_offset'])))
model.schedule_departure_times=Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['departure_dt_parsed'])))

    'Any'. The default domain for Param objects is 'Any'.  However, we will be
    changing that default to 'Reals' in the future.  If you really intend the
    domain of this Paramto be 'Any', you can suppress this warning by
    explicitly specifying 'within=Any' to the Param constructor.  (deprecated
    in 5.6.9, will be removed in (or after) 6.0) (called from c:\Users\Harsh
    Vora\AppData\Local\Programs\Python\Python310\lib\site-
    packages\pyomo\core\base\indexed_component.py:633)
    'Any'. The default domain for Param objects is 'Any'.  However, we will be
    changing that default to 'Reals' in the future.  If you really intend the
    domain of this Paramto be 'Any', you can suppress this warning by
    explicitly specifying 'within=Any' to the Param constructor.  (deprecated
    in 5.6.9, will be removed in (or after) 6.0) (called from c:\Users\Harsh
    Vora\AppData\Local\Programs\Python\Python310\lib\site-
    packages\pyomo\core\base\indexed_component.py:633)
    impl

In [16]:
# Variables
model.x = Var(model.SHIPMENTS, model.LINEHAULS, domain=Binary)
x=model.x

In [17]:
# Trade-off parameter (0 <= alpha <= 1)
alpha = 0.5  # You can adjust this value to balance between cost and OTP

In [18]:
# Function to calculate LTL cost
def calculate_ltl_cost(weight, rate_break):
    applicable_rate = max(rate_break.keys())
    for threshold, rate in sorted(rate_break.items()):
        if weight <= threshold:
            applicable_rate = rate
            break
    return weight * applicable_rate

In [19]:
def obj_rule(model):
    cost_term = sum(model.x[s, l] * (model.weight[s] / capacity.loc[0, 'max_weight_kgs']) * schedule['parsed_rate_break'][l] 
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'FTL') + \
                sum(model.x[s, l] * calculate_ltl_cost(model.weight[s], schedule['parsed_rate_break'][l])
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'LTL')
    
    #in hours
    otp_term = sum(model.x[s, l] * (model.due_time[s] - (model.ready_time[s] + timedelta(hours=int(model.travel_time[l])))).total_seconds() / 3600 
                   for s in model.SHIPMENTS for l in model.LINEHAULS if (model.due_time[s] - (model.ready_time[s] + timedelta(hours=int(model.travel_time[l])))).total_seconds() >= 0)
    
    return (1 - alpha) * cost_term - alpha * otp_term

model.obj = Objective(rule=obj_rule, sense=minimize)

In [20]:
# Constraints
def route_constraint(model, s):
    return sum(model.x[s, l] for l in model.LINEHAULS) == 1
model.route_con = Constraint(model.SHIPMENTS, rule=route_constraint)

In [21]:
def weight_constraint(model):
    return sum(model.weight[s] * model.x[s, l] for s in model.SHIPMENTS for l in model.LINEHAULS) <= capacity.loc[0, 'max_weight_kgs']
model.weight_con = Constraint(rule=weight_constraint)

In [22]:
def pallet_constraint(model):
    return sum(model.pallet[s] * model.x[s, l] for s in model.SHIPMENTS for l in model.LINEHAULS) <= capacity.loc[0, 'max_pallet_count']
model.pallet_con = Constraint(rule=pallet_constraint)

In [23]:
shipment_ready_time[1].timestamp()

1715263200.0

In [24]:
schedule['departure_dt_parsed'][0].timestamp()

1715025600.0

In [25]:
# Departure time constraint
def departure_time_constraint(model, s, l):
    ready_time = model.ready_time[s]
    #departure_datetime=schedule.loc[l, 'departure_dt_parsed']
    departure_datetime=model.schedule_departure_times[l]
    
    #departure_day = schedule.loc[l, 'day_of_week']
    #departure_time = schedule.loc[l, 'parsed_departure_time']
    
    # Calculate the departure datetime
    #ready_time_dt = pd.to_datetime(ready_time, unit='s')
    #departure_datetime = datetime.combine(ready_time_dt.date(), departure_time) + timedelta(days=(departure_day - ready_time_dt.weekday() - 1) % 7)
    
    return departure_datetime.timestamp() > ready_time.timestamp()
model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)

ERROR: Rule failed when generating expression for Constraint
    departure_time_con4 with index (1, 0): ValueError: Invalid constraint
    expression. The constraint expression resolved to a trivial Boolean
    (False) instead of a Pyomo object. Please modify your rule to return
    Constraint.Infeasible instead of False.

    Error thrown for Constraint 'departure_time_con4[1,0]'
ERROR: Constructing component 'departure_time_con4' from data=None failed:
    ValueError: Invalid constraint expression. The constraint expression
    resolved to a trivial Boolean (False) instead of a Pyomo object. Please
    modify your rule to return Constraint.Infeasible instead of False.

    Error thrown for Constraint 'departure_time_con4[1,0]'


ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (False) instead of a Pyomo object. Please modify your rule to return Constraint.Infeasible instead of False.

Error thrown for Constraint 'departure_time_con4[1,0]'

In [ ]:
def departure_time_constraint(model, s, l):
    ready_time = model.ready_time[s]
    departure_day = schedule.loc[l, 'day_of_week']
    departure_time = schedule.loc[l, 'parsed_departure_time']
    
    # Calculate the departure datetime
    ready_time_dt = pd.to_datetime(ready_time, unit='s')
    departure_datetime = datetime.combine(ready_time_dt.date(), departure_time) + timedelta(days=int((departure_day - ready_time_dt.weekday() - 1)) % 7)
    
    return departure_datetime.timestamp() >= ready_time
model.departure_time_con3 = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)


In [ ]:
# # Due time constraint
def due_time_constraint(model, s):
    return model.due_time[s] >= sum(model.ready_time[s] + model.travel_time[l] * 3600 * model.x[s, l] for l in model.LINEHAULS)
model.due_time_con = Constraint(model.SHIPMENTS, rule=due_time_constraint)

In [ ]:
# Due time constraint
#def due_time_constraint(model, s):
#    return model.due_time[s] >= sum(model.ready_time[s] + timedelta(hours=int(model.travel_time[l])) * model.x[s, l] for l in model.LINEHAULS)
#model.due_time_con = Constraint(model.SHIPMENTS, rule=due_time_constraint)

In [ ]:
# # Parameters in the model
# model.weight = Param(model.SHIPMENTS, initialize=shipment_weight)
# model.pallet = Param(model.SHIPMENTS, initialize=shipment_pallet)
# model.ready_time = Param(model.SHIPMENTS, initialize=shipment_ready_time)
# model.due_time = Param(model.SHIPMENTS, initialize=shipment_due_time)
# model.travel_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['travel_hrs_with_timezone_offset'])))

In [ ]:
schedule['travel_hrs_with_timezone_offset']

In [ ]:
a=shipment_ready_time[1]+timedelta(hours=int(schedule['travel_hrs_with_timezone_offset'][0]))

In [ ]:
shipment_due_time[1]

In [ ]:
shipment_due_time[1]>a

In [ ]:
# # Departure time constraint
# def departure_time_constraint(model, s, l):
#     ready_time = model.ready_time[s]
#     day_of_week = schedule.loc[l, 'day_of_week']
#     departure_time = schedule.loc[l, 'parsed_departure_time']
    
#     ready_time_dt = pd.to_datetime(ready_time, unit='s')
#     departure_datetime = datetime.combine(ready_time_dt.date(), departure_time) + timedelta(days=int(day_of_week - ready_time_dt.weekday()) % 7)
    
#     if departure_datetime.timestamp() >= ready_time:
#         return model.x[s, l] == 1
#     return Constraint.Skip

# model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)

In [ ]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

In [ ]:
print(results)